[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/sdxl-colab/blob/main/sdxl_v1.0_diffusers_train_dreambooth_lora_colab.ipynb)

In [ ]:
!pip install xformers bitsandbytes transformers accelerate -q
!pip install git+https://github.com/huggingface/diffusers.git -q
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py -O /content/train_dreambooth_lora_sdxl.py
!accelerate config default

# !mkdir /content/images
# !pip install datasets
# from datasets import load_dataset
# dataset = load_dataset("camenduru/plushies", split="train")
# for idx, image in enumerate(dataset['image']):
#   image.save(f"/content/images/{idx}.jpg")

# !wget https://huggingface.co/test/resolve/main/test.zip -O /content/test.zip
# !mkdir /content/images
# !unzip /content/test.zip
# !mv /content/*.jpg /content/images

In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --instance_data_dir="images" \
  --output_dir="lora" \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of a plushie toy" \
  --resolution=1024 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --enable_xformers_memory_efficient_attention \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --max_train_steps=500 \
  --checkpointing_steps=717 \
  --seed="0"

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", vae=vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
pipe.load_lora_weights("/content/lora", weight_name="pytorch_lora_weights.bin")

In [ ]:
while True:
  prompt = "a photo of a plushie toy"
  negative_prompt = ""
  import random
  random = random.randint(0, 9223372036854775807)
  generator = torch.Generator("cuda").manual_seed(random)
  image = pipe(prompt=prompt, negative_prompt=negative_prompt, generator=generator, num_inference_steps=20).images[0]
  image.save(f"/content/output/{random}.jpg")
  # from IPython.display import Image
  # Image(f"/content/output/{random}.jpg")

In [ ]:
del pipe
refiner = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0", variant="fp16", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

In [ ]:
rimage = refiner(prompt=prompt, negative_prompt=negative_prompt, generator=generator, num_inference_steps=20, image=image).images[0]
rimage